1. __Cargamos las librerias Phython__

In [1]:
import urllib3 #Libreria para descargar URL
import re #Libreria para exprecion regular

In [13]:
# download: funcion para descargar el sitio web
# parametros in:
# url: direccion del sitio web
# num_retries: numeros de reintentos de la solicitud en caso de error
# user_agent: informacion de la peticion por defecto el valor 'wswp' (Estandar web scraping con Python)
def download(url,user_agent='wswp',num_retries=2):
    print ('Downloading:', url)
    
    headers = {'User-agent': user_agent}
    
    http = urllib3.PoolManager()
    response = http.request('GET', url, retries=False,headers=headers)
    
    if response.status == 200:
        html = response.data
    else:
        print('Download error: cod_status => ', response.status)
        html = None
        if num_retries > 0:            
            if 500 <= response.status < 600:                
                # retry 5XX HTTP errors                
                return download(url, user_agent, num_retries-1)
    
    return html.decode('utf-8')

__Descargamos HTML del sitio web__

In [38]:
url = 'https://www.corabastos.com.co/sitio/historicoApp2/reportes/prueba.php'
html = download(url)
# print("html => ", html)

Downloading: https://www.corabastos.com.co/sitio/historicoApp2/reportes/prueba.php


/home/datamining/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


__Extraer los datos del HTML__

Buscamos los datos en el HTML por medio de patrones utilizando expreciones regulares

In [39]:
array_tags = re.findall('<img src="../img/logosolo.gif" align="bottom" />(.*?)<br />.*?<div class="table-responsive">(.*?)</div><b>', html,flags=re.DOTALL)